# --- Day 9: Movie Theater ---
https://adventofcode.com/2025/day/9

In [12]:
from collections import defaultdict

def getTestInput():
	with open("sampleInput.txt") as file:
		return file.read()
	
def getInput():
	with open("redTiles.txt") as file:
		return file.read()

In [13]:
redTiles = getInput().splitlines()

def getArea(corner1: str, corner2: str) -> int:
	"""Gets the area of a rectangle given two opposite corners"""
	x1, y1 = [int(x) for x in corner1.split(",")]
	x2, y2 = [int(x) for x in corner2.split(",")]
	return abs(x1-x2+1) * abs(y1-y2+1)

# Loop through each corner pair
largestArea = 0
for i, tile1 in enumerate(redTiles):
	for tile2 in redTiles[i+1:]:
		area = getArea(tile1, tile2)
		largestArea = area if area > largestArea else largestArea

print(f"Largest rectangle area: {largestArea}")

Largest rectangle area: 4782896435


# --- Part Two ---

In [168]:
Tile = tuple[int, int] # Type alias for a tile
redTiles = getInput().splitlines()
redTiles = getTestInput().splitlines()
redTiles = [tuple(map(int, x.split(","))) for x in redTiles]
redTiles.append(redTiles[0]) # Make sure we loop around

def printFloorPlan(borders, xMax, yMax):
	"""TEST OUTPUT"""
	for x in range(xMax):
		for y in range(yMax):
			if (y,x) in borders:
				print("X", end="")
			else:
				print(".", end="")
		print()
		
def getArea(tile1: Tile, tile2: Tile) -> int:
	"""Gets the area of a rectangle given two opposite corners"""
	x1, y1 = tile1
	x2, y2 = tile2
	return abs(x1-x2+1) * abs(y1-y2+1)

def getBorder(tile1: Tile, tile2: Tile) -> list[Tile]:
	"""Get a border from 2 tiles"""
	x1, y1 = tile1
	x2, y2 = tile2
	if x1 == x2:
		return set([(x1, y) for y in range(min(y1, y2), max(y1,y2)+1)])
	return set([(x, y1) for x in range(min(x1, x2), max(x1,x2)+1)])

def getAdjacentTiles(tile: Tile) -> list[Tile]:
	"""Get all 8 adjacent tiles"""
	tileChanges = [[-1,-1], [-1,0], [-1,1], [0,-1], [0,1], [1,-1], [1,0], [1,1]]
	tileX, tileY = tile
	return [(tileX + xChange, tileY + yChange) for xChange, yChange in tileChanges]

def getTileScore(tile: Tile, borders: set[Tile]) -> int:
	"""Get all 8 adjacent tiles and return how many are part of the border"""	
	return len([x for x in getAdjacentTiles(tile) if x in borders])

def getStartTile(borders: set[Tile]):
	"""Get the first outer border tile by moving diagonally until it hits a border"""
	xy = 0
	while True:
		xy += 1
		if getTileScore((xy,xy), borders) > 0:
			return (xy, xy)

def getOuterBorder(borders: set[Tile]) -> set[Tile]:
	"""Find outer border using dfs"""
	startTile = getStartTile(borders)
	stack = [startTile]
	explored = set([startTile])

	# Continue looping while there are still tiles to be explored
	while stack:
		currentTile = stack.pop()
		
		# Get 8 surrounding coords, check to see if it is an outer border meaning:
		# It has not already been seen, it is not a border itself, and
		# it is connected to at least one other border
		for successor in getAdjacentTiles(currentTile):
			if successor not in explored and successor not in borders and getTileScore(successor, borders) > 0:
				stack.append(successor)
				explored.add(successor)
	
	return explored

borders = set()
for i in range(len(redTiles) - 1):
	borders |= getBorder(redTiles[i], redTiles[i + 1])

outerBorder = getOuterBorder(borders)
printFloorPlan(borders | outerBorder, 10, 15)

......XXXXXXX..
......XXXXXXX..
.XXXXXXX...XX..
.XXXXXXX...XX..
.XX........XX..
.XXXXXXXXX.XX..
.XXXXXXXXX.XX..
........XXXXX..
........XXXXX..
...............


In [149]:
Tile = tuple[int, int] # Type alias for a tile
redTiles = getInput().splitlines()
#redTiles = getTestInput().splitlines()
redTiles = [tuple(map(int, x.split(","))) for x in redTiles]
redTiles.append(redTiles[0])

xCoords = sorted(list({x[0] for x in redTiles}))
yCoords =  sorted(list({x[1] for x in redTiles}))
xCompression = {num: compressedNum + 1 for compressedNum, num in enumerate(xCoords)}
yCompression = {num: compressedNum + 1 for compressedNum, num in enumerate(yCoords)}
xDecompression = {compressedNum + 1: num for compressedNum, num in enumerate(xCoords)}
yDecompression = {compressedNum + 1: num for compressedNum, num in enumerate(yCoords)}

def printFloorPlan(tiles: list[Tile], xMax: int, yMax: int):
	"""TEST OUTPUT"""
	for x in range(xMax):
		for y in range(yMax):
			if (y,x) in tiles:
				print("X", end="")
			else:
				print(".", end="")
		print()

def compressCoords(tile: Tile, xCompression: dict[int, int], yCompression: dict[int, int]) -> Tile:
	return (xCompression[tile[0]], yCompression[tile[1]])

def getArea(tile1: Tile, tile2: Tile) -> int:
	"""Gets the area of a rectangle given two opposite corners"""
	x1, y1 = tile1
	x2, y2 = tile2
	return abs(x1-x2+1) * abs(y1-y2+1)

def getBorder(tile1: Tile, tile2: Tile) -> list[Tile]:
	"""Get a border from 2 tiles"""
	x1, y1 = tile1
	x2, y2 = tile2
	if x1 == x2:
		return [(x1, y) for y in range(min(y1, y2), max(y1,y2)+1)]
	return [(x, y1) for x in range(min(x1, x2), max(x1,x2)+1)]

def floodFill(borders: list[Tile] | set[Tile], start: Tile, limit: int) -> set[Tile]:
	stack = [start]
	explored = set(start)

	while stack:
		# If we've gone over our guess of how big the space will be return an empty set
		if len(explored) > limit:
			return set()
		
		currentTile = stack.pop()
		surroundingCoords = [
			(currentTile[0] - 1, currentTile[1]),
			(currentTile[0] + 1, currentTile[1]),
			(currentTile[0], currentTile[1] - 1),
			(currentTile[0], currentTile[1] + 1)]
		
		# Loop through all surrounding coordinates and add them to the stack if they've not yet been seen
		for successor in surroundingCoords:
			if successor not in explored and successor not in borders:
				stack.append(successor)
				explored.add(successor)
	
	return explored

def getAllRedGreenTiles(borders: set[Tile], redTiles: set[Tile]) -> set[Tile]:
	# Start with over estimation of the size of the inside tiles
	limit = ((max({x[0] for x in compressedBorders}) - min({x[0] for x in compressedBorders}) + 1) *
			(max({x[1] for x in compressedBorders}) - min({x[1] for x in compressedBorders}) + 1))

	# Find any green tile (non corner border)
	greenTiles = list(borders - redTiles)

	for greenTile in greenTiles:
		startTiles = [(greenTile[0] - 1, greenTile[1]),
			(greenTile[0] + 1, greenTile[1]),
			(greenTile[0], greenTile[1] - 1),
			(greenTile[0], greenTile[1] + 1)]

		# Loop through potential start points
		for start in startTiles:
			if start in greenTiles: continue
			# Skip if we're starting on a border
			if start in compressedBorders: continue
			# If we successfully complete the flood fill, return union of borders and new green tiles
			if greenTiles := floodFill(compressedBorders, start, limit):
				return borders | greenTiles
		
def getAllPairs(redTiles: set[Tile]) -> list[tuple[Tile, Tile]]:
	"""Gets every pair of red tiles"""
	pairs = []
	for i, tile1 in enumerate(redTiles):
		for tile2 in redTiles[i+1:]:
			pairs.append((tile1, tile2))

	return pairs

def getRectangleTiles(tile1: Tile, tile2: Tile):
	tiles = set()
	minX, maxX = min([tile1[0], tile2[0]]), max([tile1[0], tile2[0]])
	minY, maxY = min([tile1[1], tile2[1]]), max([tile1[1], tile2[1]])
	for x in range(minX, maxX + 1):
		for y in range(minY, maxY + 1):
			tiles.add((x,y))
			
	return tiles

def isValidRectangle(tile1: Tile, tile2: Tile, allRedGreenTiles: set[Tile]):
	minX, maxX = min([tile1[0], tile2[0]]), max([tile1[0], tile2[0]])
	minY, maxY = min([tile1[1], tile2[1]]), max([tile1[1], tile2[1]])
	for x in range(minX, maxX + 1):
		for y in range(minY, maxY + 1):
			if (x,y) not in allRedGreenTiles:
				return False
			
	return True

compressedBorders = set()
for i in range(len(redTiles) - 1):
	tile1 = compressCoords(redTiles[i], xCompression, yCompression)
	tile2 = compressCoords(redTiles[i + 1], xCompression, yCompression)
	compressedBorders |= set(getBorder(tile1, tile2))

# Loop through all pairs of compressed red tiles
compressedRedTiles = [compressCoords(tile, xCompression, yCompression) for tile in redTiles]
allRedGreenTiles = getAllRedGreenTiles(compressedBorders, set(compressedRedTiles))
largestValidRectangleArea = 0

for tile1, tile2 in getAllPairs(compressedRedTiles):
	# Decompress tiles before getting the area
	decompressedTile1 = compressCoords(tile1, xDecompression, yDecompression)
	decompressedTile2 = compressCoords(tile2, xDecompression, yDecompression)
	area = getArea(decompressedTile1, decompressedTile2)

	# If we've found a new largest valid area then update largestValidRectangleArea
	if area > largestValidRectangleArea and isValidRectangle(tile1, tile2, allRedGreenTiles):
		largestValidRectangleArea = area

print(f"Largest valid rectangle area: {largestValidRectangleArea}")

Largest valid rectangle area: 1540025792


### Cool ideas that aren't working
1. Get the borders around the original borders (outer borders), if the perimeter of a rectangle hits that border at all then it is invalid (I think this would work, but in it's current state does not) (In case I ever try picking this up again: try starting from 0,0 and moving diagonally until you hit a border, then use that as your start for the outer border)
2. Check to see if the inner rectangle contains any borders (Should work, I have no idea why it gives the wrong answer)

In [ ]:
#redTiles = getInput().splitlines()
redTiles = getTestInput().splitlines()
redTiles = [tuple(map(int, x.split(","))) for x in redTiles]
redTiles.append(redTiles[0]) # Make sure we loop around

def printFloorPlan(redTiles, borders, outerBorder, xMax, yMax):
	"""TEST OUTPUT"""
	for x in range(xMax):
		for y in range(yMax):
			if (y,x) in redTiles:
				print("#", end="")
			elif (y,x) in borders:
				print(".", end="")
			elif (y,x) in outerBorder:
				print("@", end="")
			else:
				print(".", end="")
		print()
		
def getArea(corner1: str, corner2: str) -> int:
	"""Gets the area of a rectangle given two opposite corners"""
	x1, y1 = [int(x) for x in corner1.split(",")]
	x2, y2 = [int(x) for x in corner2.split(",")]
	return abs(x1-x2+1) * abs(y1-y2+1)

def getBorder(tile1: list[int, int], tile2: list[int, int]) -> list[list[int, int]]:
	"""Get a border from 2 tiles"""
	x1, y1 = tile1
	x2, y2 = tile2
	if x1 == x2:
		return set([(x1, y) for y in range(min(y1, y2), max(y1,y2)+1)])
	return set([(x, y1) for x in range(min(x1, x2), max(x1,x2)+1)])

def getPossibleOuterBorderStarts(tile1: list[int, int], tile2: list[int, int]) -> tuple[list[int, int], list[int, int], str]:
	""" 
	Takes in two tiles that are connected and returns an inner border and outer border start coordinate e.g.
	....O.....
	..XXXXX...
	....I.....
	Finds the middle of the border and gets coords on either side (does not know which is which)
	"""
	x1, y1 = tile1
	x2, y2 = tile2
	xMid = (x1 + x2) // 2
	yMid = (y1 + y2) // 2
	if x1 == x2:
		return [x1 + 1, yMid], [x1 - 1, yMid], "right"
	return [xMid, y1 + 1], [xMid, y1 - 1], "up"

def getOuterBorder(x: int, y: int, direction: str) -> set[tuple[int, int]]:
	"""Find outer border using dfs.
	Must start moving clockwise"""
	# If we've already visited this one, return explored
	stack = [(x, y, direction)]
	explored = set()

	while stack:
		x, y, direction = stack.pop()

		# If we hit a border then that means it's the inner boundary, so return an empty set
		if (x,y) in borders:
			return set()

		right = (x,y+1)
		left = (x,y-1)
		up = (x-1,y)
		down = (x+1,y)
		
		# If moving right and blocked right, move up
		if direction == "right" and right in borders:
			if up not in explored:
				stack.append((up[0], up[1], "up"))
				explored.add(up)
		# If moving right and nothing below, move down
		elif direction == "right" and down not in borders:
			if down not in explored:
				stack.append((down[0], down[1], "down"))
				explored.add(down)
		# If moving right and no blockers, just move right
		elif direction == "right":
			if right not in explored:
				stack.append((right[0], right[1], "right"))
				explored.add(right)

		# If moving left and blocked left, move down
		elif direction == "left" and left in borders:
			if down not in explored:
				stack.append((down[0], down[1], "down"))
				explored.add(down)
		# If moving left and nothing above, move up
		elif direction == "left" and up not in borders:
			if up not in explored:
				stack.append((up[0], up[1], "up"))
				explored.add(up)
		# If moving left and no blockers, just move left
		elif direction == "left":
			if left not in explored:
				stack.append((left[0], left[1], "left"))
				explored.add(left)

		# If moving up and blocked up, move left
		elif direction == "up" and up in borders:
			if left not in explored:
				stack.append((left[0], left[1], "left"))
				explored.add(left)
		# If moving up and nothing right, move right
		elif direction == "up" and right not in borders:
			if right not in explored:
				stack.append((right[0], right[1], "right"))
				explored.add(right)
		# If moving up and no blockers, just move up
		elif direction == "up":
			if up not in explored:
				stack.append((up[0], up[1], "up"))
				explored.add(up)

		# If moving down and blocked down, move right
		elif direction == "down" and down in borders:
			if right not in explored:
				stack.append((right[0], right[1], "right"))
				explored.add(right)
		# If moving down and nothing left, move left
		elif direction == "down" and left not in borders:
			if left not in explored:
				stack.append((left[0], left[1], "left"))
				explored.add(left)
		# If moving down and no blockers, just move down
		elif direction == "down":
			if down not in explored:
				stack.append((down[0], down[1], "down"))
				explored.add(down)
	
	return explored

borders = set()
for i in range(len(redTiles) - 1):
	borders |= getBorder(redTiles[i], redTiles[i + 1])

possibleOuter1, possibleOuter2, direction = getPossibleOuterBorderStarts(redTiles[0],redTiles[1])
print(possibleOuter1, possibleOuter2, direction)
print(f"Start point: {possibleOuter1}, Direction: {direction}, Border length: {len(getOuterBorder(possibleOuter1[0], possibleOuter1[1], direction))}")
printFloorPlan(redTiles, borders, getOuterBorder(*possibleOuter1, direction), 9, 14) # TEST
print(f"Start point: {possibleOuter2}, Direction: {direction}, Border length: {len(getOuterBorder(possibleOuter2[0], possibleOuter2[1], direction))}")
printFloorPlan(redTiles, borders, getOuterBorder(*possibleOuter2, direction), 9, 14) # TEST

In [ ]:
Tile = tuple[int, int] # Type alias for a tile
#redTiles = getInput().splitlines()
redTiles = getTestInput().splitlines()
redTiles = [tuple(map(int, x.split(","))) for x in redTiles]
redTiles.append(redTiles[0]) # Make sure we loop around

def getArea(tile1: Tile, tile2: Tile) -> int:
	"""Gets the area of a rectangle given two opposite corners"""
	x1, y1 = tile1
	x2, y2 = tile2
	return abs(x1-x2+1) * abs(y1-y2+1)

def getBorder(tile1: Tile, tile2: Tile) -> list[Tile]:
	"""Get a border from 2 tiles"""
	x1, y1 = tile1
	x2, y2 = tile2
	if x1 == x2:
		return [(x1, y) for y in range(min(y1, y2), max(y1,y2)+1)]
	return [(x, y1) for x in range(min(x1, x2), max(x1,x2)+1)]

def getShortestSides(tile1: Tile, tile2: Tile) -> tuple[list[Tile], list[Tile]]:
	x1, y1 = tile1
	x2, y2 = tile2
	xRangeSmaller = abs(x1-x2) > abs(y1-y2)
	# Get the two smallest sides
	if xRangeSmaller:
		side1 = getBorder((x1, y1), (x1, y2))
		side2 = getBorder((x2, y1), (x2, y2))
	else:
		side1 = getBorder((x1, y1), (x2, y1))
		side2 = getBorder((x2, y2), (x1, y2))
	
	return side1, side2

def isValidRectangle(tile1: Tile, tile2: Tile, borderDict: dict[str, list[Tile]]):
	side1, side2 = getShortestSides(tile1, tile2)

	for tile1, tile2 in list(zip(side1, side2))[1:-1]:
		# If X's are the same between sides, check columns to see if there 
		# are any borders in the area of the rectangle 
		if tile1[0] == tile2[0]:
			minY, maxY = min([tile1[1], tile2[1]]), max([tile1[1], tile2[1]])
			borderInArea = any([minY < tile[1] and tile[1] < maxY for tile in borderDict[f"X{tile1[0]}"]])
			if borderInArea:
				return False
		
		# If Y's are the same between sides, check rows to see if there 
		# are any borders in the area of the rectangle
		elif tile1[1] == tile2[1]:
			minX, maxX = min([tile1[0], tile2[0]]), max([tile1[0], tile2[0]])
			borderInArea = any([minX < tile[0] and tile[0] < maxX for tile in borderDict[f"Y{tile1[1]}"]])
			if borderInArea:
				return False
		else:
			print("Something has gone horribly wrong")
	
	return True

# I know this looks stupid but its used for quick lookups on a specific axis
borderDict = defaultdict(set)
for i in range(len(redTiles) - 1):
	for tile in getBorder(redTiles[i], redTiles[i + 1]):
		borderDict[f"X{tile[0]}"].add(tile)
		borderDict[f"Y{tile[1]}"].add(tile)

largestValidArea = 0
for i, tile1 in enumerate(redTiles):
	for tile2 in redTiles[i+1:]:
		area = getArea(tile1, tile2)
		if area > largestValidArea and isValidRectangle(tile1, tile2, borderDict):
			largestValidArea = area

print(f"Largest valid rectangle area: {largestValidArea}")